# Simulation of microservices

This simulation attempts to balance the size of a micro service

## Microservice

The size of a [microservice](https://en.wikipedia.org/wiki/Microservices) is related to the number of features it provides.

In [34]:
from typing import List, Tuple, Set
from fractions import Fraction

In [35]:
class MicroService:
    def __init__(self):
        self.feature_count = 0
    
    def set_feature_count(self, count: int):
        self.feature_count = count
        return self
        
    def __str__(self):
        return "MicroService: features: {}".format(self.feature_count)
           

In [36]:
micro1 = MicroService()
micro1.set_feature_count(3)
print(micro1)

MicroService: features: 3


## ServiceSuite

A service suite represents an aggregation of services used by a department.



In [37]:
class ServiceSuite:
    def __init__(self):
        self.services = []
           
    def __str__(self):
        return "ServiceSuite: count: {}, features: {}".format(self.get_service_count(), self.get_feature_count())
        
    def add_microservice(self, microservice: MicroService):
        self.services.append(microservice)
        return self
        
    def get_feature_count(self)->int:
        return sum([s.feature_count for s in self.services])
    
    def get_service_count(self)->int:
        return len(self.services)
    
    def get_summary(self)->str:
        features_by_service = " ".join([str(s.feature_count) for s in self.services])
        return "{} services, features: {}".format(self.get_service_count(), features_by_service)
 
   

In [38]:
serviceSuite1 = ServiceSuite()
serviceSuite1.add_microservice(MicroService().set_feature_count(4))
serviceSuite1.add_microservice(MicroService().set_feature_count(3))
print(serviceSuite1)
print(serviceSuite1.get_summary())

ServiceSuite: count: 2, features: 7
2 services, features: 4 3


## Specifications

### Developement time

The developer time in hours

In [39]:
class DevTimeSpecs:
    def __init__(self):
        self.hours_per_service = 1
        self.hours_per_feature = 1
        self.penalty_exp = 1.1
    
    def set_hours_per_service(self, hours: float):
        self.hours_per_service = hours
        return self

    def set_hours_per_feature(self, hours: float):
        self.hours_per_feature = hours
        return self
    
    def set_penalty_exp(self, penalty_exp: float):
        self.penalty_exp = penalty_exp
        return self
   
    def __str__(self):
        return "DevTimeSpecs: h/service : {}, h/feature: {}".format(self.hours_per_service, self.hours_per_feature)
        
    def get_devtime(self, service: MicroService)->float:
        return self.hours_per_service + service.feature_count*(self.penalty_exp**service.feature_count)*self.hours_per_feature
   
    def get_devtime_for_suite(self, suite: ServiceSuite)->float:
        return sum([self.get_devtime(service) for service in suite.services])


### Business specifications

We can simplify the business specifications as a number of features to provide.

In [40]:
class BusinessSpecs:
    def __init__(self):
        self.feature_count = 0
        self.min_features_by_service = 3
        self.dev_time_specs = None
    
    def set_feature_count(self, count: int):
        self.feature_count = count
        return self
    
    def set_dev_time_specs(self, dev_time_specs: DevTimeSpecs):
        self.dev_time_specs = dev_time_specs
        return self
    
    def set_min_features_by_service(self, min_features_by_service: int):
        self.min_features_by_service = min_features_by_service
        return self
       
    def __str__(self):
        return "BusinessSpecs: features: {}, min feat./serv. {}, dev time: {}".format(self.feature_count, self.min_features_by_service, self.dev_time_specs)
    

In [41]:
businessSpecs = BusinessSpecs()
businessSpecs.set_feature_count(100)
businessSpecs.set_dev_time_specs(DevTimeSpecs().set_hours_per_service(8).set_hours_per_feature(16).set_penalty_exp(1.05))
print("hours", businessSpecs.dev_time_specs.get_devtime(MicroService().set_feature_count(4)))

hours 85.79240000000001


## Simulation

The library Pandas(https://pandas.pydata.org/) makes it simpler to manipulate data.

In [55]:
import pandas as pd

class Simulation:
    def __init__(self):
        self.business_specs = None
        self.service_suite_list = []
        
    def set_business_specs(self, business_specs: BusinessSpecs):
        self.business_specs = business_specs
        return self
    
    def add_service_suite(self, service_suite: ServiceSuite):
        self.service_suite_list.append(service_suite)
        return self
    
    def _to_suite_tuple(self, suite: ServiceSuite):
        return (suite.get_summary(), suite.get_service_count(), self.business_specs.dev_time_specs.get_devtime_for_suite(suite))
        
    def to_list(self):
        return [self._to_suite_tuple(suite) for suite in self.service_suite_list]
    
    def to_dataframe(self):
        return pd.DataFrame(self.to_list())
      

In [56]:
simulation1 = Simulation().set_business_specs(businessSpecs).add_service_suite(serviceSuite1)
print(simulation1.to_dataframe())

                           0  1         2
0  2 services, features: 4 3  2  149.3584


In [22]:
simulation_data = pd.DataFrame({'a': list('CCCDDDEEE'),
                     'b': [2, 7, 4, 1, 2, 6, 8, 4, 7]})
print(simulation_data)

   a  b
0  C  2
1  C  7
2  C  4
3  D  1
4  D  2
5  D  6
6  E  8
7  E  4
8  E  7


### Visualisation

[Altair](https://altair-viz.github.io/getting_started/overview.html) is a declarative statistical visualization library for Python, based on Vega and Vega-Lite that will need to be [installed](https://altair-viz.github.io/getting_started/installation.html).

In [23]:
import altair as alt

chart = alt.Chart(simulation_data)
alt.Chart(simulation_data).mark_point()
alt.Chart(simulation_data).mark_point().encode(
    x='a',
)
alt.Chart(simulation_data).mark_point().encode(
    x='a',
    y='b'
)


alt.Chart(...)